In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name
%load_ext jupyter_black
spark = SparkSession.builder.appName("project1").getOrCreate()

The jupyter_black extension is already loaded. To reload it, use:
  %reload_ext jupyter_black
23/01/08 22:45:54 WARN Utils: Your hostname, zechchair-gl553vd resolves to a loopback address: 127.0.1.1; using 192.168.0.130 instead (on interface wlp2s0)
23/01/08 22:45:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/08 22:45:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [46]:
from os import listdir
from os.path import isfile, join

my_path = "/home/zechchair/Documents/github/M2-data-science/big data /project/input"
only_files = [my_path + "/" + f for f in listdir(my_path) if isfile(join(my_path, f))]
files_names = [f for f in listdir(my_path) if isfile(join(my_path, f))]
onlyCities_names = set(
    [files_name.split(".")[0].split("_")[0] for files_name in files_names]
)

In [125]:
from pyspark.sql.functions import split, reverse, when, lit, avg, sum

In [141]:
df = spark.read.format("csv").load(my_path + "/*.txt")

# Add a column with the file name
df = df.withColumn("city", input_file_name())
df = df.withColumn("store_N", reverse(split("city", "/"))[0])
df = df.withColumn("city", split("store_N", "\\.")[0])
df = df.withColumn("city", split("city", "_")[0])
df = df.withColumn("income", df["_c0"].substr(5, 7))
df = df.withColumn("month", df["_c0"].substr(0, 3))
df.show()

+------+----------+----------------+------+-----+
|   _c0|      city|         store_N|income|month|
+------+----------+----------------+------+-----+
|JAN 13|      lyon|        lyon.txt|    13|  JAN|
|FEB 12|      lyon|        lyon.txt|    12|  FEB|
|MAR 14|      lyon|        lyon.txt|    14|  MAR|
|APR 15|      lyon|        lyon.txt|    15|  APR|
|MAY 12|      lyon|        lyon.txt|    12|  MAY|
|JUN 15|      lyon|        lyon.txt|    15|  JUN|
|JUL 19|      lyon|        lyon.txt|    19|  JUL|
|AUG 25|      lyon|        lyon.txt|    25|  AUG|
|SEP 13|      lyon|        lyon.txt|    13|  SEP|
|OCT 11|      lyon|        lyon.txt|    11|  OCT|
|NOV 22|      lyon|        lyon.txt|    22|  NOV|
|DEC 22|      lyon|        lyon.txt|    22|  DEC|
|JAN 21|marseilles|marseilles_1.txt|    21|  JAN|
|FEB 21|marseilles|marseilles_1.txt|    21|  FEB|
|MAR 21|marseilles|marseilles_1.txt|    21|  MAR|
|APR 27|marseilles|marseilles_1.txt|    27|  APR|
|MAY 25|marseilles|marseilles_1.txt|    25|  MAY|


In [142]:
df.select(avg("income")).show()

+-----------------+
|      avg(income)|
+-----------------+
|23.19871794871795|
+-----------------+



In [143]:
df.groupBy("month").agg(avg("income")).show()

+-----+------------------+
|month|       avg(income)|
+-----+------------------+
|  APR| 20.23076923076923|
|  OCT| 26.53846153846154|
|  NOV| 24.53846153846154|
|  FEB|19.153846153846153|
|  SEP| 25.53846153846154|
|  JAN| 20.76923076923077|
|  AUG|23.076923076923077|
|  MAR| 17.53846153846154|
|  DEC|              29.0|
|  JUN|27.846153846153847|
|  JUL|21.692307692307693|
|  MAY| 22.46153846153846|
+-----+------------------+



In [144]:
df.groupBy("city", "month").agg(avg("income")).orderBy("city").show()

+-----+-----+-----------+
| city|month|avg(income)|
+-----+-----+-----------+
|anger|  AUG|       15.0|
|anger|  MAY|       12.0|
|anger|  SEP|       13.0|
|anger|  APR|       15.0|
|anger|  MAR|       14.0|
|anger|  NOV|       14.0|
|anger|  JAN|       13.0|
|anger|  JUN|       15.0|
|anger|  JUL|       19.0|
|anger|  DEC|       16.0|
|anger|  OCT|        8.0|
|anger|  FEB|       12.0|
| lyon|  MAR|       14.0|
| lyon|  AUG|       25.0|
| lyon|  APR|       15.0|
| lyon|  MAY|       12.0|
| lyon|  OCT|       11.0|
| lyon|  JUL|       19.0|
| lyon|  FEB|       12.0|
| lyon|  JUN|       15.0|
+-----+-----+-----------+
only showing top 20 rows



In [145]:
df.groupBy("city").agg(avg("income")).orderBy("city").show()

+----------+------------------+
|      city|       avg(income)|
+----------+------------------+
|     anger|13.833333333333334|
|      lyon|16.083333333333332|
|marseilles|21.458333333333332|
|    nantes|             17.25|
|      nice|16.916666666666668|
|    orlean|16.333333333333332|
|     paris| 43.55555555555556|
|    rennes|              15.0|
|  toulouse|             14.75|
|    troyes|17.833333333333332|
+----------+------------------+



In [146]:
df.groupBy("city").agg(avg("income")).orderBy("city").show()

+----------+------------------+
|      city|       avg(income)|
+----------+------------------+
|     anger|13.833333333333334|
|      lyon|16.083333333333332|
|marseilles|21.458333333333332|
|    nantes|             17.25|
|      nice|16.916666666666668|
|    orlean|16.333333333333332|
|     paris| 43.55555555555556|
|    rennes|              15.0|
|  toulouse|             14.75|
|    troyes|17.833333333333332|
+----------+------------------+



In [147]:
df.groupBy("month", "store_N").agg(sum("income")).show()

+-----+----------------+-----------+
|month|         store_N|sum(income)|
+-----+----------------+-----------+
|  DEC|        lyon.txt|       22.0|
|  SEP|marseilles_1.txt|       23.0|
|  OCT|        lyon.txt|       11.0|
|  MAY|        lyon.txt|       12.0|
|  JAN|marseilles_1.txt|       21.0|
|  FEB|marseilles_1.txt|       21.0|
|  APR|        lyon.txt|       15.0|
|  MAR|marseilles_1.txt|       21.0|
|  APR|marseilles_1.txt|       27.0|
|  AUG|marseilles_1.txt|       22.0|
|  OCT|marseilles_1.txt|       28.0|
|  NOV|        lyon.txt|       22.0|
|  JUL|        lyon.txt|       19.0|
|  JUN|        lyon.txt|       15.0|
|  JUL|marseilles_1.txt|       21.0|
|  JUN|marseilles_1.txt|       25.0|
|  MAR|        lyon.txt|       14.0|
|  MAY|marseilles_1.txt|       25.0|
|  NOV|marseilles_1.txt|       24.0|
|  DEC|marseilles_1.txt|       26.0|
+-----+----------------+-----------+
only showing top 20 rows



In [148]:
df.groupBy("month", "store_N").agg(avg("income")).show()

+-----+----------------+-----------+
|month|         store_N|avg(income)|
+-----+----------------+-----------+
|  DEC|        lyon.txt|       22.0|
|  SEP|marseilles_1.txt|       23.0|
|  OCT|        lyon.txt|       11.0|
|  MAY|        lyon.txt|       12.0|
|  JAN|marseilles_1.txt|       21.0|
|  FEB|marseilles_1.txt|       21.0|
|  APR|        lyon.txt|       15.0|
|  MAR|marseilles_1.txt|       21.0|
|  APR|marseilles_1.txt|       27.0|
|  AUG|marseilles_1.txt|       22.0|
|  OCT|marseilles_1.txt|       28.0|
|  NOV|        lyon.txt|       22.0|
|  JUL|        lyon.txt|       19.0|
|  JUN|        lyon.txt|       15.0|
|  JUL|marseilles_1.txt|       21.0|
|  JUN|marseilles_1.txt|       25.0|
|  MAR|        lyon.txt|       14.0|
|  MAY|marseilles_1.txt|       25.0|
|  NOV|marseilles_1.txt|       24.0|
|  DEC|marseilles_1.txt|       26.0|
+-----+----------------+-----------+
only showing top 20 rows



In [160]:
from pyspark.sql.functions import split, reverse, when, lit, avg, sum, max, col

# Group the data by the month column
grouped_df = df.groupBy("month")

# Find the maximum income for each group
max_df = grouped_df.agg(max(col("income")).alias("max_income"))
max_df = max_df.withColumnRenamed("month", "Month_max")


# Join the original data with the aggregated data on the month and income columns
joined_df = df.join(
    max_df,
    (df["month"] == max_df["Month_max"]) & (df["income"] == max_df["max_income"]),
)

# Select the month, store, and max_income columns
result_df = joined_df.select("month", "store_N", "max_income")
result_df.show()

+-----+-----------+----------+
|month|    store_N|max_income|
+-----+-----------+----------+
|  APR| rennes.txt|         9|
|  APR|   nice.txt|         9|
|  AUG|paris_2.txt|        45|
|  DEC|paris_1.txt|        71|
|  FEB|paris_2.txt|        42|
|  JAN|paris_1.txt|        51|
|  JUL|paris_1.txt|        61|
|  JUN|paris_2.txt|        85|
|  MAR|paris_2.txt|        44|
|  MAY|paris_2.txt|        72|
|  NOV|paris_2.txt|        64|
|  OCT| orlean.txt|         8|
|  OCT|  anger.txt|         8|
|  SEP|paris_2.txt|        63|
+-----+-----------+----------+



In [1]:
%load_ext jupyter_black


In [1]:
import socket
import sys
import requests
import requests_oauthlib
import json

# def create_twitter_stream(ssc):
# Replace these with your own access tokens
CONSUMER_KEY = "J32dJ5o0oyt4XobzJ32yE7igX"
CONSUMER_SECRET = "pH9P9d1sqtJv2BBCrh8MXnx19vaSWaMiqp9yYD42ZTayMZkzZB"
ACCESS_TOKEN = "1208952534662078464-CDWpV6ZFH0B2gvMnpXHWcsZaXRn4Rh"
ACCESS_TOKEN_SECRET = "JvEKx3ZcAEnaKzjF2R8cCFYVsmHm9xkBBbYFiSYc90lsc"
my_auth = requests_oauthlib.OAuth1(
    CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET
)


def get_tweets():
    url = "https://stream.twitter.com/1.1/statuses/filter.json"
    query_data = [("language", "en"), ("locations", "-130,-20,100,50"), ("track", "#")]
    query_url = url + "?" + "&".join([str(t[0]) + "=" + str(t[1]) for t in query_data])
    response = requests.get(query_url, auth=my_auth, stream=True)
    print(query_url, response)
    return response


def send_tweets_to_spark(http_resp, tcp_connection):
    for line in http_resp.iter_lines():
        try:
            full_tweet = json.loads(line)
            tweet_text = full_tweet["text"]
            print("Tweet Text: " + tweet_text)
            print("------------------------------------------")
            tcp_connection.send(tweet_text + "\n")
        except:
            e = sys.exc_info()[0]
            print("Error: %s" % e)


TCP_IP = "localhost"
TCP_PORT = 9009
conn = None
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.bind((TCP_IP, TCP_PORT))
s.listen(1)
print("Waiting for TCP connection...")
conn, addr = s.accept()
print("Connected... Starting getting tweets.")
resp = get_tweets()
send_tweets_to_spark(resp, conn)

# we create this class that inherits from the StreamListener in tweepy StreamListener
# class TweetListener(tweepy.Stream): # NEW CODE

#     def __init__(self, csocket):
#         self.client_socket = csocket
#     # we override the on_data() function in StreamListener
#     def on_data(self, data):
#         try:
#             message = json.loads( data )
#             print( message['text'].encode('utf-8') )
#             self.client_socket.send( message['text'].encode('utf-8') )
#             return True
#         except BaseException as e:
#             print("Error on_data: %s" % str(e))
#         return True

#     def if_error(self, status):
#         print(status)
#         return True

Waiting for TCP connection...


KeyboardInterrupt: 

In [3]:
from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row,SQLContext
import sys
import requests
# create spark configuration
conf = SparkConf()
conf.setAppName("TwitterStreamApp")
# create spark context with the above configuration
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
# create the Streaming Context from the above spark context with interval size 2 seconds
ssc = StreamingContext(sc, 2)
# setting a checkpoint to allow RDD recovery
ssc.checkpoint("checkpoint_TwitterApp")
# read data from port 9009
dataStream = ssc.socketTextStream("localhost",9009)

23/01/09 01:56:34 WARN Utils: Your hostname, zechchair-gl553vd resolves to a loopback address: 127.0.1.1; using 192.168.0.130 instead (on interface wlp2s0)
23/01/09 01:56:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/09 01:56:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/home/zechchair/anaconda3/envs/optimisation/lib/python3.9/site-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [4]:
# split each tweet into words
words = dataStream.flatMap(lambda line: line.split(" "))
# filter the words to get only hashtags, then map each hashtag to be a pair of (hashtag,1)
hashtags = words.filter(lambda w: '#' in w).map(lambda x: (x, 1))
# adding the count of each hashtag to its last count
tags_totals = hashtags.updateStateByKey(aggregate_tags_count)
# do processing for each RDD generated in each interval
tags_totals.foreachRDD(process_rdd)
# start the streaming computation
ssc.start()
# wait for the streaming to finish
ssc.awaitTermination()

In [6]:
def aggregate_tags_count(new_values, total_sum):
	return sum(new_values) + (total_sum or 0)


AttributeError: 'DStream' object has no attribute 'show'

In [ ]:
def get_sql_context_instance(spark_context):
    if ('sqlContextSingletonInstance' not in globals()):
        globals()['sqlContextSingletonInstance'] = SQLContext(spark_context)
        return globals()['sqlContextSingletonInstance']
def process_rdd(time, rdd):
    print("----------- %s -----------" % str(time))
    try:
        sql_context = get_sql_context_instance(rdd.context)
        # convert the RDD to Row RDD
        row_rdd = rdd.map(lambda w: Row(hashtag=w[0], hashtag_count=w[1]))
        # create a DF from the Row RDD
        hashtags_df = sql_context.createDataFrame(row_rdd)
        # Register the dataframe as table
        hashtags_df.registerTempTable("hashtags")
        # get the top 10 hashtags from the table using SQL and print them
        hashtag_counts_df = sql_context.sql("select hashtag, hashtag_count from hashtags order by hashtag_count desc limit 10")
        hashtag_counts_df.show()
        # call this method to prepare top 10 hashtags DF and send them
        send_df_to_dashboard(hashtag_counts_df)
    except:
        e = sys.exc_info()[0]
        print("Error: %s" % e)

In [ ]:
def send_df_to_dashboard(df):
	# extract the hashtags from dataframe and convert them into array
	top_tags = [str(t.hashtag) for t in df.select("hashtag").collect()]
	# extract the counts from dataframe and convert them into array
	tags_count = [p.hashtag_count for p in df.select("hashtag_count").collect()]
	# initialize and send the data through REST API
	url = 'http://localhost:5001/updateData'
	request_data = {'label': str(top_tags), 'data': str(tags_count)}
	response = requests.post(url, data=request_data)

In [7]:
import tweepy

class TweetListener(tweepy.Stream): # NEW CODE

    def __init__(self, csocket):
        self.client_socket = csocket
    # we override the on_data() function in StreamListener
    def on_data(self, data):
        try:
            message = json.loads( data )
            print( message['text'].encode('utf-8') )
            self.client_socket.send( message['text'].encode('utf-8') )
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    def if_error(self, status):
        print(status)
        return True

def create_twitter_stream(ssc):
  # Replace these with your own access tokens
    CONSUMER_KEY    = "DHwxvBBBebmi1l4NID1Zj3Zgh"
    CONSUMER_SECRET = "LZwdVVHn2UWsQQsZAFil6m6v8UHA2PEnYJEKmmZi8tBNK82Akl"
    ACCESS_TOKEN       = "1208952534662078464-liYjvgXoLMHFjdi90DaJAPYSaxP7dB"
    ACCESS_TOKEN_SECRET= "WIKf8GvUreMpJmbYov1fDnFX8x8wUqpU6FGO2S6fKcTJT"

    # Set up the Twitter API client
    # auth = tweepy.OAuth1UserHandler(CONSUMER_KEY, CONSUMER_SECRET,
    #                                 ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
    # api = tweepy.API(auth)

    # # Create a DStream of tweets using the Twitter API
    # tweets_dstream = tweepy.Stream(auth=api.auth, listener=tweepy.TweepyUtils.TweetReceiver())
    auth = OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

    twitter_stream = Stream(auth, TweetsListener(c_socket))
    twitter_stream.filter(track=['football']) #we are interested in this topic.
  return tweets_dstream


In [2]:
from __future__ import print_function
import sys
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# Create a SparkContext and a StreamingContext
sc = SparkContext(appName="PopularHashtags")


23/01/09 02:01:46 WARN Utils: Your hostname, zechchair-gl553vd resolves to a loopback address: 127.0.1.1; using 192.168.0.130 instead (on interface wlp2s0)
23/01/09 02:01:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/09 02:01:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


AttributeError: module 'tweepy' has no attribute 'TweepyUtils'

In [8]:
ssc = StreamingContext(sc, 10)

# Create a DStream of tweets using the create_twitter_stream function
tweets_dstream = create_twitter_stream(ssc)

# Extract the hashtags from each tweet
hashtags_dstream = tweets_dstream.flatMap(lambda tweet: tweet.hashtags)

# Count the number of occurrences of each hashtag over a sliding window of 10 minutes
hashtag_counts_dstream = hashtags_dstream.map(lambda hashtag: (hashtag, 1)).reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 600, 10)

# Sort the RDD by the count in descending order and take the top 10 hashtags
top_hashtags_dstream = hashtag_counts_dstream.transform(lambda rdd: rdd.sortBy(lambda x: x[1], ascending=False).take(10))

# Print the top hashtags to the console
top_hashtags_dstream.foreachRDD(lambda rdd: print("\nTop hashtags:", rdd.collect()))

# Start the streaming context and wait for termination
ssc.start()
ssc.awaitTermination()

AttributeError: module 'tweepy' has no attribute 'TweepyUtils'

In [ ]:
import matplotlib.pyplot as plt

# Create the plot
fig, ax = plt.subplots()

# Set up a timer to update the plot every 10 seconds
def update_plot(hashtags_counts_rdd):
  # Get the top 10 hashtags and their counts
  top_hashtags = hashtags_counts_rdd.take(10)
  hashtags = [x[0] for x in top_hashtags]
  counts = [x[1] for x in top_hashtags]

  # Update the plot with the new data
  ax.clear()
  ax.bar(hashtags, counts)
  plt.xticks(rotation=90)
  fig.canvas.draw()

hashtags_counts.foreachRDD(update_plot)

plt.show()

# Start the streaming process
ssc.start()

# Wait for the streaming to finish
ssc.awaitTermination()
